<a href="https://colab.research.google.com/github/shaking54/CS114.K21.KHTN/blob/master/Sarcasm_Headlines.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#*Sarcasm Dectection
https://www.kaggle.com/rmisra/news-headlines-dataset-for-sarcasm-detection

- Input bài toán: Một chuỗi string

- Output: Chuỗi string nhận được có mang tính Sarcastic hay không

####*Thu thập dữ liệu

- Dữ liệu cũ về Sarcasm dectection được lấy từ Twitters. Tuy nhiên, do dữ liệu này có độ nhiễu nên vẫn còn hạn chế. Để vượt qua được điểm này thì dữ liệu mới về Sarcasm detection được lấy từ 2 websites đó là https://www.theonion.com/ và https://www.huffpost.com . Về dữ liệu mang tính Sarcastic thì được lấy từ trang web TheOnion ở trong mục News in Brief và News in Photos categories. Còn dữ liệu không mang tính Sarcastic thì được lấy từ trang web HuffPost.


In [66]:
import pandas as pd
data=pd.read_json('/content/drive/My Drive/Sarcasm_Headlines_Dataset.json', lines=True)
data.head()

,article_link,headline,is_sarcastic
0,https://www.huffingtonpost.com/entry/versace-b...,former versace store clerk sues over secret 'b...,0
1,https://www.huffingtonpost.com/entry/roseanne-...,the 'roseanne' revival catches up to our thorn...,0
2,https://local.theonion.com/mom-starting-to-fea...,mom starting to fear son's web series closest ...,1
3,https://politics.theonion.com/boehner-just-wan...,"boehner just wants wife to listen, not come up...",1
4,https://www.huffingtonpost.com/entry/jk-rowlin...,j.k. rowling wishes snape happy birthday in th...,0


In [67]:
sarcasm = data.drop(columns='article_link')

##*Xử lý dữ liệu

- Các headline sẽ được chuyển về lowercase, loại bỏ các ký tự đặc biệt.
- Sau đó sẽ phân chia dữ liệu train và dữ liệu test
- Sử dụng Word2vec để trích xuất đặc trưng.
- Khai báo Vectorize để biểu diễn dữ liệu train dưới dạng vector và tạo một từ điển từ dữ liệu train
- Sử dụng lại từ điển của dữ liệu trên để biểu diễn dữ liệu test dưới dạng vector

In [68]:
import re
def pre_process(headline):
	headline = headline.lower()                                               
	headline = re.sub("&lt;/?.*?&gt;"," &lt;&gt; ", headline)     
	headline = re.sub("(\\d|\\W)+"," ", headline)                   
	return headline

sarcasm['headline'] = sarcasm['headline'].apply(lambda x:pre_process(x))

In [69]:
from sklearn.model_selection import train_test_split
X = sarcasm['headline']
y = sarcasm['is_sarcastic']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)

In [70]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

vectorizer1 = CountVectorizer(analyzer='word',ngram_range=(1,1))
X_train = vectorizer1.fit_transform(X_train)

vectorizer2 = CountVectorizer(vocabulary=vectorizer1.vocabulary_,ngram_range=(1,1))
X_test = vectorizer2.fit_transform(X_test)

##Chọn Model để train dữ liệu
- Sử dụng các thuật toán như Suport vector classifications, Decision Tree classifications, Random Forest Classification, Voting Classification.


In [71]:
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
SVClassifiers = SVC()
SVClassifiers.fit(X_train, y_train)
SVC_predict = SVClassifiers.predict(X_test)
SCV_score = accuracy_score(y_test, SVC_predict)
SCV_score

0.840134780980906

In [72]:
from sklearn.tree import DecisionTreeClassifier

Decisionclassifier = DecisionTreeClassifier()
Decisionclassifier.fit(X_train, y_train)
Decisionclassifier_predict = Decisionclassifier.predict(X_test)
Decisionclassifier_score = accuracy_score(y_test, Decisionclassifier_predict)
Decisionclassifier_score

0.7538999126419568

In [73]:
from sklearn.ensemble import RandomForestClassifier
RFClassifier = RandomForestClassifier()
RFClassifier.fit(X_train, y_train)
RFClassifier_predict = RFClassifier.predict(X_test)
RFClassifier_score = accuracy_score(y_test, RFClassifier_predict)
RFClassifier_score

0.8090602770497941

In [74]:
# from sklearn.ensemble import VotingClassifier
# from sklearn.svm import SVC
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.tree import DecisionTreeClassifier
# RFClassifier = RandomForestClassifier()
# SVClassifiers = SVC()
# Decisionclassifier = DecisionTreeClassifier()
# voting_clf = VotingClassifier(
#         estimators=[('svc', clf), ('rf', random_clf), ('dc', tree_clf)],
#         voting='hard'
# )

#* Chạy Model đã train với dữ liệu mới tạo được
- Model Support vector machine classifications
    - performance được 80.9% so với trên dữ liệu test là xấp xỉ 83%

- Model Decision Tree classifications
    - Performance được 68.5 % so với xấp xỉ 75% với dữ liệu test

- Model Random Forest Classifications
    - Performance được 73.3 % so với xấp xỉ 81% trên dữ liệu test



In [90]:
newdata = pd.read_json('/content/drive/My Drive/headline_data.json', lines=True)
newdata

,headline,is_sarcastic
0,ExxonMobil Simplifies Oil Extraction By Cuttin...,1
1,Quaker Oats Replaces Historically Racist Aunt ...,1
2,Showrunner Worried This A Bad Time For New Ser...,1
3,Anna Kendrick’s ‘Love Life’ Pulled From HBO Ma...,1
4,South Secedes From NASCAR Following Confederat...,1
...,...,...
1995,Susan Rice Explains Exactly Why Trump's White ...,0
1996,Scathing Supercut Shows How Quickly Trump Flip...,0
1997,Trump's 'Racist' Toddler Video Removed By Face...,0
1998,"50 Years After MLK, Pastors Lead A New Poor Pe...",0


In [91]:
newdata['headline'] = newdata['headline'].apply(lambda x:pre_process(x))
Xtest = newdata['headline']
ytest = newdata['is_sarcastic']

Xtest = vectorizer2.fit_transform(Xtest)

In [92]:
for cls in [SVClassifiers, Decisionclassifier,RFClassifier]:
    ypred = cls.predict(Xtest)
    yscore = accuracy_score(ytest, ypred)
    print(cls.__class__.__name__, yscore)

SVC 0.806
DecisionTreeClassifier 0.685
RandomForestClassifier 0.733


In [96]:
headline = input("Enter headline: ")
for cls in [SVClassifiers, Decisionclassifier,RFClassifier]:
    print(cls.__class__.__name__, cls.predict(vectorizer2.transform([pre_process(headline)])))

Enter headline: After Weeks Of Anticipation, Trump Rally Crowd Underwhelms
SVC [0]
DecisionTreeClassifier [0]
RandomForestClassifier [0]
